In [1]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
tf.__version__

'2.3.1'

In [6]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   vertical_flip = True)

In [9]:
training_set = train_datagen.flow_from_directory('Dataset/Train',
                                                 target_size = (100, 100),
                                                 class_mode = 'binary')

# Preprocessing the Test set
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('Dataset/Test',
                                            target_size = (100, 100),
                                            class_mode = 'binary')

Found 45 images belonging to 2 classes.
Found 12 images belonging to 2 classes.


### Lets Build Convolution Network

In [10]:
from tensorflow.keras.layers import Conv2D

In [13]:
# Part 2 - Building the CNN

# Initialising the CNN
cnn = tf.keras.models.Sequential()

# Step 1 - Convolution
cnn.add(tf.keras.layers.Conv2D(filters=32,padding="same",kernel_size=3, activation='relu', input_shape=[100,100, 3]))

# Step 2 - Pooling
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Adding a second convolutional layer
cnn.add(tf.keras.layers.Conv2D(filters=32,padding='same',kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Step 3 - Flattening
cnn.add(tf.keras.layers.Flatten())

# Step 4 - Full Connection
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

# Step 5 - Output Layer
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [14]:
cnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 100, 100, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 50, 50, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 50, 50, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 25, 25, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 20000)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               2560128   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 1

In [15]:
# Part 3 - Training the CNN

# Compiling the CNN
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Training the CNN on the Training set and evaluating it on the Test set
cnn.fit(x = training_set, validation_data = test_set, epochs = 10)

Epoch 1/10
2/2 [==============================] - 2s 991ms/step - loss: 1.2133 - accuracy: 0.4444 - val_loss: 1.3257 - val_accuracy: 0.5000
Epoch 2/10
2/2 [==============================] - 1s 357ms/step - loss: 1.0817 - accuracy: 0.5333 - val_loss: 0.6606 - val_accuracy: 0.5000
Epoch 3/10
2/2 [==============================] - 2s 787ms/step - loss: 0.7425 - accuracy: 0.3556 - val_loss: 0.6852 - val_accuracy: 0.5000
Epoch 4/10
2/2 [==============================] - 2s 822ms/step - loss: 0.7201 - accuracy: 0.4667 - val_loss: 0.6512 - val_accuracy: 0.5833
Epoch 5/10
2/2 [==============================] - 2s 778ms/step - loss: 0.6735 - accuracy: 0.5111 - val_loss: 0.5932 - val_accuracy: 0.9167
Epoch 6/10
2/2 [==============================] - 2s 796ms/step - loss: 0.6519 - accuracy: 0.6222 - val_loss: 0.6502 - val_accuracy: 0.5000
Epoch 7/10
2/2 [==============================] - 1s 292ms/step - loss: 0.6519 - accuracy: 0.5556 - val_loss: 0.5713 - val_accuracy: 0.9167
Epoch 8/10
2/2 [====

In [16]:
# save it as a h5 file


from tensorflow.keras.models import load_model

cnn.save('model_rmale_female.h5')

In [17]:
from tensorflow.keras.models import load_model
 
# load model
model = load_model('model_rmale_female.h5')

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 100, 100, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 50, 50, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 50, 50, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 25, 25, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 20000)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               2560128   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 1

In [20]:
# Part 4 - Making a single prediction

import numpy as np
from tensorflow.keras.preprocessing import image
test_image = image.load_img('Dataset/Test/male/download (6).jpg', target_size = (100,100))
test_image = image.img_to_array(test_image)
test_image=test_image/255
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)

In [22]:
test_image1 = image.load_img('Dataset/Test/female/download (6).jpg', target_size = (100,100))
test_image1 = image.img_to_array(test_image1)
test_image1=test_image1/255
test_image1 = np.expand_dims(test_image1, axis = 0)
result1 = cnn.predict(test_image1)

In [21]:
result    # as it is Binary prob so above 50% will be  male as arranged by folder wise as male folder is 2nd

array([[0.75080895]], dtype=float32)

In [24]:
result1     # as it is Binary prob so below 50% will be  female as arranged by folder wise as female folder is 1st

array([[0.47219095]], dtype=float32)

In [26]:
if result[0]<=0.5:
    print("The image classified is female")
else:
    print("The image classified is male")

The image classified is male
